<a href="https://colab.research.google.com/github/MadmanMarble/Planet_U-NET/blob/main/Multiclass_U_Net_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 49.5 MB/s eta 0:00:00


In [4]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras.utils import normalize
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
import rasterio
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
import tensorflow as tf
from tensorflow.keras.models import load_model

In [5]:
# Assign image and label variables
x_training_01 = sorted(glob.glob(f"/content/drive/MyDrive/data/images/x_training_140/1year_140/*.tif"))
x_training_02 = sorted(glob.glob(f"/content/drive/MyDrive/data/images/x_training_140/2year_140/*.tif"))
x_training_03 = sorted(glob.glob(f"/content/drive/MyDrive/data/images/x_training_140/3year_140/*.tif"))
y_training = sorted(glob.glob(f"/content/drive/MyDrive/data/labels/y_training_140/*.tif"))

# Split into training and testing datasets (80% train, 20% test)
x_train_01, x_test_01, y_train, y_test = train_test_split(x_training_01, y_training, test_size=0.22, random_state=9)
x_train_02, x_test_02, y_train, y_test = train_test_split(x_training_02, y_training, test_size=0.22, random_state=9)
x_train_03, x_test_03, y_train, y_test = train_test_split(x_training_03, y_training, test_size=0.22, random_state=9)

In [6]:
# Download the training and testing dataset to local location

# Load the training labels
destination = '/content/y_train'
os.makedirs(destination, exist_ok=True)
for file in y_train:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/y_train"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")

# Load the test labels
destination = '/content/y_test'
os.makedirs(destination, exist_ok=True)
for file in y_test:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/y_test"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")


# Load the training images for experiment one
destination = '/content/x_train_01'
os.makedirs(destination, exist_ok=True)
for file in x_train_01:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/x_train_01"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")

# Load the test images for experiment one
destination = '/content/x_test_01'
os.makedirs(destination, exist_ok=True)
for file in x_test_01:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/x_test_01"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")


# Load the training images for experiment two
destination = '/content/x_train_02'
os.makedirs(destination, exist_ok=True)
for file in x_train_02:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/x_train_02"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")

# Load the test images for experiment two
destination = '/content/x_test_02'
os.makedirs(destination, exist_ok=True)
for file in x_test_02:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/x_test_02"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")


# Load the training images for experiment three
destination = '/content/x_train_03'
os.makedirs(destination, exist_ok=True)
for file in x_train_03:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/x_train_03"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")

# Load the test images for experiment three
destination = '/content/x_test_03'
os.makedirs(destination, exist_ok=True)
for file in x_test_03:
    dest_file = os.path.join(destination, os.path.basename(file))
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')
directory_path = "/content/x_test_03"
entries = os.listdir(directory_path)
num_files = len(entries)
print(f"There are {num_files} files in {directory_path}.")

Copied /content/drive/MyDrive/data/labels/y_training_140/797_multi_wl.tif to /content/y_train/797_multi_wl.tif
Copied /content/drive/MyDrive/data/labels/y_training_140/142_multi_wl.tif to /content/y_train/142_multi_wl.tif
Copied /content/drive/MyDrive/data/labels/y_training_140/170_multi_wl.tif to /content/y_train/170_multi_wl.tif
Copied /content/drive/MyDrive/data/labels/y_training_140/221_multi_wl.tif to /content/y_train/221_multi_wl.tif
Copied /content/drive/MyDrive/data/labels/y_training_140/778_multi_wl.tif to /content/y_train/778_multi_wl.tif
Copied /content/drive/MyDrive/data/labels/y_training_140/795_multi_wl.tif to /content/y_train/795_multi_wl.tif
Copied /content/drive/MyDrive/data/labels/y_training_140/1580_multi_wl.tif to /content/y_train/1580_multi_wl.tif
Copied /content/drive/MyDrive/data/labels/y_training_140/1529_multi_wl.tif to /content/y_train/1529_multi_wl.tif
Copied /content/drive/MyDrive/data/labels/y_training_140/143_multi_wl.tif to /content/y_train/143_multi_wl.t

In [7]:
# Assign variables to each dataset set on the local location
y_train = sorted(glob.glob(f"/content/y_train/*.tif"))
y_test= sorted(glob.glob(f"/content/y_test/*.tif"))

x_train_01 = sorted(glob.glob(f"/content/x_train_01/*.tif"))
x_test_01 = sorted(glob.glob(f"/content/x_test_01/*.tif"))

x_train_02 = sorted(glob.glob(f"/content/x_train_02/*.tif"))
x_test_02 = sorted(glob.glob(f"/content/x_test_02/*.tif"))

x_train_03 = sorted(glob.glob(f"/content/x_train_03/*.tif"))
x_test_03 = sorted(glob.glob(f"/content/x_test_03/*.tif"))

In [8]:
# Define data loader for label data
def read_image_labels(file_path):
    with rasterio.open(file_path) as src:
      label_image = src.read(1)
      return label_image.astype(np.int32)

In [9]:
# Define data loader for one year of images
def read_multi_image_withfive(file_path):
    with rasterio.open(file_path) as src:
      bands = [src.read(i) for i in range(1,5)]
      image = np.stack(bands, axis=-1)
      image = image.astype(float)
      max_value = image.max(axis=(0,1), keepdims=True)
      min_value = image.min(axis=(0,1), keepdims=True)
      image = (image - min_value) / (max_value - min_value)
      band_five = src.read(5)
      band_five = band_five.astype(float) /band_five.max()
      band_five = np.expand_dims(band_five, axis=-1)
      combined_image = np.concatenate([image, band_five], axis=-1)
      return combined_image

In [10]:
# Define data loader for two years of images
def read_nine_band_image(file_path):
    with rasterio.open(file_path) as src:
      bands = [src.read(i) for i in range(1,9)]
      image = np.stack(bands, axis=-1)
      image = image.astype(float)
      max_value = image.max(axis=(0,1), keepdims=True)
      min_value = image.min(axis=(0,1), keepdims=True)
      image = (image - min_value) / (max_value - min_value)
      band_five = src.read(9)
      band_five = band_five.astype(float) /band_five.max()
      band_five = np.expand_dims(band_five, axis=-1)
      combined_image = np.concatenate([image, band_five], axis=-1)
      return combined_image

In [11]:
# Define data loader for three years of images
def read_thirteen_band_image(file_path):
    with rasterio.open(file_path) as src:
      bands = [src.read(i) for i in range(1,13)]
      image = np.stack(bands, axis=-1)
      image = image.astype(float)
      max_value = image.max(axis=(0,1), keepdims=True)
      min_value = image.min(axis=(0,1), keepdims=True)
      image = (image - min_value) / (max_value - min_value)
      band_five = src.read(13)
      band_five = band_five.astype(float) /band_five.max()
      band_five = np.expand_dims(band_five, axis=-1)
      combined_image = np.concatenate([image, band_five], axis=-1)
      return combined_image



In [12]:
# Assign training data and test data variables
y_train_data = [read_image_labels(file_path) for file_path in y_train]
y_test_data = [read_image_labels(file_path) for file_path in y_test]

x_train_01_data = [read_multi_image_withfive(file_path) for file_path in x_train_01]
x_test_01_data = [read_multi_image_withfive(file_path) for file_path in x_test_01]

x_train_02_data = [read_nine_band_image(file_path) for file_path in x_train_02]
x_test_02_data = [read_nine_band_image(file_path) for file_path in x_test_02]

x_train_03_data = [read_thirteen_band_image(file_path) for file_path in x_train_03]
x_test_03_data = [read_thirteen_band_image(file_path) for file_path in x_test_03]

In [13]:
# Convert the training data and test data for each experiment from a list to a NumPy array
x_train_01 = np.array(x_train_01_data)
x_test_01 = np.array(x_test_01_data)

x_train_02 = np.array(x_train_02_data)
x_test_02 = np.array(x_test_02_data)

x_train_03 = np.array(x_train_03_data)
x_test_03 = np.array(x_test_03_data)

y_train = np.array(y_train_data)
y_test = np.array(y_test_data)

In [14]:
# Substract 1 from the label training data and label test data so the range starts at 0.
# Subtract 1 from each label in the training dataset. This is commonly done to shift
# label indices from a range that starts at 1 to a range that starts at 0, which is
# standard in many machine learning frameworks that expect zero-indexed labels.
y_train_shifted = y_train -1
y_test_shifted = y_test -1

In [15]:
# Convert the shifted training labels into one-hot encoded format. `to_categorical`
# function is used to create a binary matrix representation of the labels.
# `num_classes=7` indicates that there are seven different classes, ensuring
# that the output matrix columns correspond to each class.
y_train_encoded = to_categorical(y_train_shifted, num_classes = 7)
y_test_encoded = to_categorical(y_test_shifted, num_classes = 7)

In [ ]:
def unet_model(input_size=(384, 384, 13), num_classes=7):
    inputs = Input(input_size)

    # Downsampling (Contracting Path)
    c1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    c1 = BatchNormalization()(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    c2 = BatchNormalization()(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    c3 = BatchNormalization()(c3)

    # Upsampling (Expanding Path)
    u4 = UpSampling2D((2, 2))(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u4)
    c4 = BatchNormalization()(c4)
    c4 = Dropout(0.1)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    c4 = BatchNormalization()(c4)

    u5 = UpSampling2D((2, 2))(c4)
    u5 = concatenate([u5, c1], axis=3)
    c5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u5)
    c5 = BatchNormalization()(c5)
    c5 = Dropout(0.1)(c5)
    c5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    c5 = BatchNormalization()(c5)

    # Output layer
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(c5)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Create the U-Net model
model = unet_model()
print(model.summary())

In [ ]:
# Build callback to save predictions during training.
class SavePredictionCallback(Callback):
    def __init__(self, x_holdout_dir, output_folder, model_input_size, epoch_frequency=5):
        self.x_holdout_dir = glob.glob(x_holdout_dir)
        self.output_folder = output_folder
        self.model_input_size = model_input_size
        self.epoch_frequency = epoch_frequency

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.epoch_frequency == 0:
            for image_path in self.x_holdout_dir:
                with rasterio.open(image_path) as src:
                    bands = [src.read(i) for i in range(1, 13)]
                    image = np.stack(bands, axis=-1)
                    image = image.astype(float)
                    max_value = image.max(axis=(0, 1), keepdims=True)
                    min_value = image.min(axis=(0, 1), keepdims=True)
                    image = (image - min_value) / (max_value - min_value)
                    band_five = src.read(13)
                    band_five = band_five.astype(float) / band_five.max()
                    band_five = np.expand_dims(band_five, axis=-1)
                    combined_image = np.concatenate([image, band_five], axis=-1)
                    combined_image_batch = np.expand_dims(combined_image, axis=0)

                # Predict using the model
                prediction = self.model.predict(combined_image_batch)
                prediction_image = np.argmax(prediction, axis=-1)[0, :, :]

                # Prepare the output path
                output_path = os.path.join(self.output_folder, f"epoch_{epoch+1}_{os.path.basename(image_path)}")

                # Save the prediction using rasterio
                meta = src.meta.copy()
                meta.update(dtype=rasterio.uint8, count=1, compress='lzw', nodata= None)
                with rasterio.open(output_path, 'w', **meta) as dst:
                    dst.write(prediction_image.astype(rasterio.uint8), 1)

# Initialization of the callback
img_height, img_width = 384, 384
output_folder = "/content/drive/MyDrive/data/model_prediction/threeyear_data/20240418_1448"
os.makedirs(output_folder, exist_ok=True)
x_holdout_dir = "/content/drive/MyDrive/data/images/x_test/3year/*.tif" # Change this based on which experiment is running.

savepred_callback = SavePredictionCallback(
    x_holdout_dir=x_holdout_dir,
    output_folder=output_folder,
    model_input_size = (img_height, img_width),
    epoch_frequency=5
)


In [ ]:
# Initialize the Adam optimizer with a specific learning rate
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile the model with the optimizer, loss function, and
model.compile(optimizer = adam_optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy', 'categorical_accuracy'])
callbacks = [keras.callbacks.ModelCheckpoint(filepath = "/content/drive/MyDrive/data/model_prediction/threeyear_data/20240418_1448/20240418_1448_3.h5", monitor = 'val_loss', save_best_only = True), savepred_callback]

In [ ]:
# Run the model
model.fit(three_x_train, y_train_encoded, batch_size=8, epochs=500, validation_data=(three_x_test, y_test_encoded),
          callbacks=[callbacks])

Predict using each model


In [19]:
import os
import glob
import numpy as np
import rasterio
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import Callback

class PredictWetlandClasses:
    def __init__(self, model_path, x_holdout_dir, output_folder, model_input_size):
        self.model = load_model(model_path)
        self.x_holdout_dir = glob.glob(x_holdout_dir)
        self.output_folder = output_folder
        self.model_input_size = model_input_size

    def predict(self):
        for image_path in self.x_holdout_dir:
            with rasterio.open(image_path) as src:
                bands = [src.read(i) for i in range(1, 6)]  # Read all 13 bands
                image = np.stack(bands[:-1], axis=-1)  # Stack first 12 bands
                image = image.astype(float)
                max_value = image.max(axis=(0, 1), keepdims=True)
                min_value = image.min(axis=(0, 1), keepdims=True)
                image = (image - min_value) / (max_value - min_value)

                band_thirteen = bands[-1].astype(float) / bands[-1].max()
                band_thirteen = np.expand_dims(band_thirteen, axis=-1)
                combined_image = np.concatenate([image, band_thirteen], axis=-1)
                combined_image_batch = np.expand_dims(combined_image, axis=0)

                # Predict using the loaded model
                prediction = self.model.predict(combined_image_batch)
                prediction_image = np.argmax(prediction, axis=-1)[0, :, :]

                # Prepare the output path
                output_path = os.path.join(self.output_folder, f"predicted_{os.path.basename(image_path)}")

                # Save the prediction using rasterio
                meta = src.meta.copy()
                meta.update(dtype=rasterio.uint8, count=1, compress='lzw')
                with rasterio.open(output_path, 'w', **meta) as dst:
                    dst.write(prediction_image.astype(rasterio.uint8), 1)

# Example usage
model_path = '/content/drive/MyDrive/data/model_prediction/oneyear_data/20240418_1448/20240418_1448.h5'  # Path to your trained model
img_height, img_width = 384, 384  # Adjust these values based on your model input size
output_folder = "/content/zzz_test"
x_holdout_dir = "/content/drive/MyDrive/data/images/x_test/1year/*.tif"  # Directory containing TIFF files

os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist
predictor = PredictWetlandClasses(
    model_path=model_path,
    x_holdout_dir=x_holdout_dir,
    output_folder=output_folder,
    model_input_size=(img_height, img_width)
)

predictor.predict()


1/1 [==============================] - 5s 5s/step


ValueError: Given nodata value, -3.3999999521443642e+38, is beyond the valid range of its data type, uint8.

In [38]:
import os
import glob
import numpy as np
import rasterio
from tensorflow.keras.models import load_model

def read_multi_image_withfive(file_path):
    with rasterio.open(file_path) as src:
        bands = [src.read(i) for i in range(1, 14)]  # Read all 5 bands, 9, or 13
        # Normalize first four bands
        image = np.stack(bands[:-1], axis=-1)
        image = image.astype(float)
        max_value = image.max(axis=(0, 1), keepdims=True)
        min_value = image.min(axis=(0, 1), keepdims=True)
        image = (image - min_value) / (max_value - min_value)

        # Normalize the fifth band (elevation)
        band_five = bands[-1].astype(float) / bands[-1].max()
        band_five = np.expand_dims(band_five, axis=-1)

        # Combine normalized bands
        combined_image = np.concatenate([image, band_five], axis=-1)
        return combined_image

class PredictWetlandClasses:
    def __init__(self, model_path, x_holdout_dir, output_folder):
        self.model = load_model(model_path)
        self.x_holdout_dir = glob.glob(x_holdout_dir)
        self.output_folder = output_folder

    def predict(self):
        for image_path in self.x_holdout_dir:
            combined_image = read_multi_image_withfive(image_path)
            combined_image_batch = np.expand_dims(combined_image, axis=0)

            # Predict using the loaded model
            prediction = self.model.predict(combined_image_batch)
            prediction_image = np.argmax(prediction, axis=-1)[0, :, :]

            # Prepare the output path
            output_path = os.path.join(self.output_folder, f"predicted_{os.path.basename(image_path)}")

            # Save the prediction using rasterio
            with rasterio.open(image_path) as src:
                meta = src.meta.copy()
                meta.update(dtype='float32', count=1, compress='lzw', nodata=-3.3999999521443642e+38)

                output_path = os.path.join(self.output_folder, f"predicted_{os.path.basename(image_path)}")
                with rasterio.open(output_path, 'w', **meta) as dst:
                  prediction_image = prediction_image.astype(meta['dtype'])  # Ensure the dtype matches meta
                  dst.write(prediction_image, 1)
                # meta.update(dtype=rasterio.uint8, count=1, compress='lzw')
                # with rasterio.open(output_path, 'w', **meta) as dst:
                #     dst.write(prediction_image.astype(rasterio.uint8), 1)

# Example usage configuration
model_path = '/content/drive/MyDrive/data/model_prediction/threeyear_data/20240418_1448/20240418_1448_3.h5'  # Path to your trained model
img_height, img_width = 384, 384  # Adjust these values based on your model input size
output_folder = "/content/threeYear_classificationMap"
x_holdout_dir = "/content/drive/MyDrive/data/images/x_test/3year/*.tif"  # Directory containing TIFF files


os.makedirs(output_folder, exist_ok=True)
predictor = PredictWetlandClasses(
    model_path=model_path,
    x_holdout_dir=x_holdout_dir,
    output_folder=output_folder
)
predictor.predict()


1/1 [==============================] - 4s 4s/step


In [39]:
import shutil

zip_name = '/content/threeYear_classificationMap'
directory_name = '/content/threeYear_classificationMap'

# Create 'path\to\zip_file.zip'
shutil.make_archive(zip_name, 'zip', directory_name)

'/content/threeYear_classificationMap.zip'

In [ ]:
from tensorflow.keras.models import load_model

# Replace 'path/to/your/model.h5' with the actual path to your saved model
model_ex01 = load_model('/content/drive/MyDrive/data/model_prediction/oneyear_data/20240418_1448/20240418_1448.h5')